In [9]:
import boto3
import pandas as pd

In [28]:
s3 = boto3.resource('s3')
bucket_name = 'mindex-data-analytics-code-challenge'
bucket = s3.Bucket(bucket_name)
for obj in bucket.objects.all():
    s3.Object(bucket_name, obj.key).download_file(f'.\\{obj.key}')
    print(obj.key)

bengals.csv
boyd_receiving.csv
chase_receiving.csv
higgins_receiving.csv


In [33]:
bengals_df = pd.read_csv('bengals.csv')
boyd_df = pd.read_csv('boyd_receiving.csv')
chase_df = pd.read_csv('chase_receiving.csv')
higgins_df = pd.read_csv('higgins_receiving.csv')